In [ ]:
%matplotlib widget
from main import *
import sympy as sp
import matplotlib.pyplot as plt
from sympy import Heaviside, sin, cos
sp.init_printing()

In [ ]:
Vcc = 5

Vin = 10 + 6 * sin(t)

In [ ]:
stabilisator = Solver.Circuit("stabilisator")
stabilisator.element(
                    VoltageSource("Vg", {"V+" : "Vg", "V-" : Gnd}, V_t = Vin),
                    
                    Resistor("Rz", {"V1" : "Vg", "V2" : "Vz"}, R = 10e3),
                    Diode("Dz", {"Vp" : "Vz", "Vn" : Gnd}, Vd = 5),
                    
                    NPN("Q1", {"Vc" : "Vg", "Vb" : "Vz", "Ve" : "Vout"}),
                    
                    Resistor("Rl", {"V1" : "Vout", "V2" : Gnd}, R = 4.4e3),
                    #CurrentSource("Iout", {"V+" : "Vout", "V-" : Gnd}, I_t = -1e-3*t),
                    
)

In [ ]:
Solver.setMain("stabilisator")
compiled = Solver.compile()

In [ ]:
print(compiled["nodes"])

Vin = compiled["voltages"]["Vg"]["V+"]
Iin = compiled["currents"]["Vg"]["V-"]
Vout = compiled["nodes"]["Vout"]
Iout = compiled["currents"]["Rl"]["V1"]

print(Vin)
print(Iin)
print(Vout)
print(Iout)

In [ ]:
try: 
    model = loadModel("stabilisator", "Laplace")
except (OSError, IOError) as e:
    model = solveLaplace(compiled, 2 * sp.pi, tstep = 0.1, measurments = {Vin, Iin, Vout, Iout}, workerN = 2, debugLog = True)
    saveModel("stabilisator", "Laplace", model)


In [ ]:
printModel(model, var=t, measurments=[Vin, Iin, Vout, Iout])

In [ ]:
measurments =   [   
                    (t, Vin, "Vin(t)"),
                    (t, Vout, "Vout(t)"),
                ]

fig, ax = plt.subplots(figsize=[15, 7])

plotTranMeasurments(model, 0, 10, 0.01, measurments, ax)
#plotMeasurments(model, -10, 10, 0.1, measurments)
#plt.legend(loc="best");
plt.grid(True);
plt.show()

In [ ]:
measurments =   [   
                    (t, 1e3 * Iin, "Iin(t)"),
                    (t, 1e3 * Iout, "Iout(t)"),
                ]

fig, ax = plt.subplots(figsize=[15, 7])

plotTranMeasurments(model, 0, 10, 0.01, measurments, ax)
#plotMeasurments(model, -10, 10, 0.1, measurments)
#plt.legend(loc="best");
plt.grid(True);
plt.show()

In [ ]:
measurments =   [   
                    (Vout, 1e3 * Iout, "Iin(t)"),
                ]

fig, ax = plt.subplots(figsize=[15, 7])

plotTranMeasurments(model, 0, 10, 0.01, measurments, ax)
#plotMeasurments(model, -10, 10, 0.1, measurments)
#plt.legend(loc="best");
plt.grid(True);
plt.show()